# Setup

In [ ]:
# Install modules with pip if not already installed

# Import dependencies

In [ ]:
import time
import csv
import re

import pandas as pd
import openpyxl
from openpyxl.styles import Font, Alignment
import google.generativeai as genai
from google.api_core import exceptions

from requests.exceptions import ReadTimeout, Timeout

# Configure API key and model

In [ ]:
# For use with Google generative AI API
genai.configure(api_key="API_KEY") # Replace API_KEY with your API key
model = genai.GenerativeModel('gemini-2.0-pro-exp-02-05') # Or another model

# Get data

In [ ]:
# Read an excel file
df = pd.read_excel("use_cases.xlsx") # Replace with your file name
use_cases = df.to_dict(orient='records')

# Run the model prompts and generate responses

## Generate each use case

In [ ]:
# Generate each use case with retry logic due to API rate limits, reduce delays and retries if needed
def generate_with_retry(model, prompt, max_retries=5, initial_delay=2, backoff_factor=2): 
    for attempt in range(max_retries):
        time.sleep(2) # Add a fixed delay just in case
        try:
            response = model.generate_content(prompt)
            return response.text  
        except (exceptions.ServiceUnavailable, exceptions.TooManyRequests, ReadTimeout, Timeout) as e:  
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt < max_retries - 1:
                delay = initial_delay * (backoff_factor ** attempt)  
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                print("Max retries reached. Returning 'N/A'")
                return "N/A"  
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            return "N/A"  

## Simulate posting on a website

In [ ]:
# Simulate posting to a website such as my.ai.se
def post_case_to_website(processed_case_data, website_url):
    
    print(f"Simulating posting case data to: {website_url}")
    print(f"Data to be posted: {processed_case_data}") 
    time.sleep(1) # Modify delay if necessary
    print("Data posted successfully.")
    print()
    
    # In a real scenario, replace with something like
    # import requests
    # response = requests.post(website_url, json=case_data)
    # if response.status_code == 200:
    #     print("Data posted successfully.")
    # else:
    #     print(f"Error posting data: {response.status_code}")

## Prompts for use cases

In [ ]:
processed_use_cases = []
for case in use_cases:
    description = case['Beskrivning']

    # Create prompts for each function call
    extract_info_prompt = f"""
    Analyze the following use case description and provide a summary of the information.  
    The company involved is a Swedish energy provider, specialising in electricity, district heating/cooling, sustainable energy.    
    In this context, "Agent" refers to an AI agent, not a human.
    Provide the information directly, without any introductory phrases or conversational language.
    Do not describe the company, the client is already familiar with it.
    Text: {description}
    """

    analysis_info_prompt = f"""
    Analyze the following text and provide the information in the following format:
    Analysis: [Analysis of the text]
    Agent Type: [Agent Type]
    Category: [Category]
    Primary Function: [Primary Function]
    Goal: [Goal]
    Business Impact: [Business Impact]
    Provide the information directly, without any introductory phrases or conversational language.
    In this context, "Agent" refers to an AI agent, not a human.   
    Text: {description}
    """
    
    ranking_info_prompt = f"""
    Priority Rank: [Priority Rank]/10, a number between 0 and 10, where 10 is the highest.
    Justification of score and reasoning for not scoring higher: [Ranking Justification]
    Provide the information directly, without any introductory phrases or conversational language.
    In this context, "Agent" refers to an AI agent, not a human.
    Text: {description}
    """
    
    feasiblilty_info_prompt = f"""
    Feasibility Score: [Feasibility Score]/10, a number between 0 and 10, where 10 is the highest.
    Justification: [Feasibility Justification]
    Provide the information directly, without any introductory phrases or conversational language.
    In this context, "Agent" refers to an AI agent, not a human.
    Text: {description}
    """
    
    integration_info_prompt = f"""
    Analyze how company-approved AI & automation tools like Power Automate, Copilot Studio, or Power Apps
    (the Microsoft Power Platform suit) can preferably be utilized in the use cases. 
    Emphasize integration needs, outlining necessary system connections and dependencies.
    Provide the information directly, without any introductory phrases or conversational language.
    In this context, "Agent" refers to an AI agent, not a human.
    Text: {description}
    """
    
    risk_info_prompt = f"""
    Conduct risk assessment and provide a summary in text format.
    Provide the information directly, without any introductory phrases or conversational language.
    In this context, "Agent" refers to an AI agent, not a human.
    Text: {description}
    """
    
    time_info_prompt = f"""
    Provide a time estimate for the implementation of the use case. If not possible to estimate, provide a reason.
    Time Estimate: [Time Estimate]
    Provide the information directly, without any introductory phrases or conversational language.
    In this context, "Agent" refers to an AI agent, not a human.
    Text: {description}
    """

    gap_info_prompt = f"""
    Identify missing information from the text and summarize it in the following format:
    Missing Information: [Missing Information]
    Questions: [Questions that need to be answered]
    Provide the information directly, without any introductory phrases or conversational language.
    In this context, "Agent" refers to an AI agent, not a human.
    Text: {description}
    """

    # Use generate_with_retry for each function call
    extracted_info_text = generate_with_retry(model, extract_info_prompt)
    print(f"Case: {case['Titel']}")
    print(f"Extracted Info: {extracted_info_text}")
    
    # Simulate posting the case card to my.ai.se
    website_url = "https://my.ai.se/use-cases"  # Dummy URL
    print("--- Case Start ---")  
    print(f"Posting to: {website_url}")
    print("Data posted:") 
    print(f"Case: {case['Titel']}")
    print() 
    
    analysis_info_text = generate_with_retry(model, analysis_info_prompt)
    print(f"{analysis_info_text}")

    ranking_info_text = generate_with_retry(model, ranking_info_prompt)
    print(f"Ranking: {ranking_info_text}")
    
    feasibility_info_text = generate_with_retry(model, feasiblilty_info_prompt)
    print(f"Feasibility: {feasibility_info_text}")
    
    integration_info_text = generate_with_retry(model, integration_info_prompt)
    print(f"Integration: {integration_info_text}")
    
    risk_info_prompt = generate_with_retry(model, risk_info_prompt)
    print(f"Risk Assessment: {risk_info_prompt}")
    
    time_info_text = generate_with_retry(model, time_info_prompt)
    print(f"Time Estimate: {time_info_text}")

    gap_info_text = generate_with_retry(model, gap_info_prompt)
    print(f"Gap Info: {gap_info_text}")
    
    time.sleep(1)
    print("Status: Data posted successfully.")
    print("--- Case End ---") # posting to website ends here

   # Append to processed_use_cases with separate columns
    processed_use_cases.append({
        'Original Title': case['Titel'],
        'Extracted Info': extracted_info_text,
        'Analysis Info': analysis_info_text,        
        'Ranking Info': ranking_info_text,
        'Feasibility Info': feasibility_info_text,
        'Integration Info': integration_info_text,
        'Risk Analysis': risk_info_prompt,
        'Time Estimate': time_info_text,
        'Gap Analysis': gap_info_text
    })
    

    print(f"Processed case: {case['Titel']}")
    print("-" * 60)
    print() # Add a new line for readability

# Export response as a spreadsheet

In [ ]:
# Create a new Excel workbook
workbook = openpyxl.Workbook()

# Create the main index sheet
index_sheet = workbook.active
index_sheet.title = "Use Case Index"
index_sheet.append(["Index", "Use Case Title", "Priority", "Feasibility"]) 

bold_font = Font(bold=True)
for cell in index_sheet[1]:
    cell.font = bold_font

# Create individual sheets for each use case
for index, case in enumerate(processed_use_cases):
    # Create a valid sheet name (replace invalid characters with underscores)
    sheet_name = case['Original Title']
    sheet_name = re.sub(r'[\\/*?\[\]:]', '', sheet_name)
    sheet_name = sheet_name[:31]
    
    # Create a new sheet
    worksheet = workbook.create_sheet(title=sheet_name)
    
    # Add data to the sheet
    worksheet.append(["Original Title", case['Original Title']])
    worksheet.append(["Extracted Info", case['Extracted Info']])
    worksheet.append(["Analysis", case['Analysis Info']])
    worksheet.append(["Ranking", case['Ranking Info']])
    worksheet.append(["Feasibility", case['Feasibility Info']])
    worksheet.append(["Integration needs", case['Integration Info']])
    worksheet.append(["Risk assessment", case['Risk Analysis']])
    worksheet.append(["Time estimate", case['Time Estimate']])
    worksheet.append(["Gap Analysis", case['Gap Analysis']])

    # Create a link to the sheet on the index page
    link_cell = index_sheet.cell(row=index_sheet.max_row + 1, column=1)
    link_cell.value = index + 1  
    link_cell = index_sheet.cell(row=index_sheet.max_row, column=2)  
    link_cell.value = case['Original Title']  
    link_cell.hyperlink = f"#'{sheet_name}'!A1"
    link_cell.style = "Hyperlink"

    ranking_text = case.get('Ranking Info', 'N/A')
    feasibility_text = case.get('Feasibility Info', 'N/A')

    # Extract numeric scores using regular expressions
    priority_match = re.search(r'Priority Rank:\s*(\d+)', ranking_text)
    feasibility_match = re.search(r'Feasibility Score:\s*(\d+)', feasibility_text)

    priority = int(priority_match.group(1)) if priority_match else 'N/A' # Convert to int if match otherwise N/A
    feasibility = int(feasibility_match.group(1)) if feasibility_match else 'N/A'

    index_sheet.cell(row=index_sheet.max_row, column=3).value = priority
    index_sheet.cell(row=index_sheet.max_row, column=4).value = feasibility

    first_column = worksheet['A'] 
    for cell in first_column:
        cell.font = bold_font

    # Apply word wrap, column width, and vertical alignment to the use case sheets
    for col in worksheet.columns:
        max_length = 0
        column = col[0].column_letter

        for cell in col:
            try:  
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass

        adjusted_width = (max_length + 2)
        if column == 'B':  # Limit width of column B
            adjusted_width = min(adjusted_width, 200)  # Adjust to your preferred max width

        worksheet.column_dimensions[column].width = adjusted_width

        for cell in col:
            cell.alignment = Alignment(wrap_text=True, vertical='top')

    # Adjust row height after all cells in the row are populated otherwise it won't show all lines
    for row in worksheet.rows:
        max_height = 15  # Default row height
        for cell in row:
            if cell.value:
                # Do some maths to estimate number of lines needed 
                lines = str(cell.value).split('\n')
                line_count = 0
                for line in lines:
                    line_count += max(1, len(line) // 80) # Adjust 80 based on your font
                cell_height = line_count * 15 
                max_height = max(max_height, cell_height)
        worksheet.row_dimensions[row[0].row].height = max_height

# Adjust column widths for the index sheet
for col in index_sheet.columns:
    max_length = 0
    column = col[0].column_letter
    
    for cell in col:
        try:  
            if len(str(cell.value)) > max_length:
                max_length = len(str(cell.value))
        except:
            pass

    adjusted_width = (max_length + 2) 
    index_sheet.column_dimensions[column].width = adjusted_width

# Remove the default sheet if it's still there
if "Sheet" in workbook.sheetnames:
    default_sheet = workbook["Sheet"]
    workbook.remove(default_sheet)

# Save the workbook to a local file
file_path = "use_cases_report.xlsx"  # Specify the desired file name
workbook.save(file_path)

print(f"Excel file created: {file_path}")

# Export response as a csv file

In [ ]:
def generate_csv_report(processed_use_cases, file_path="use_cases_report.csv"):
    
    fieldnames = [
        "Original Title",
        "Analysis Info",
        "Ranking Info",
        "Feasibility Info",
        "Integration Info",
        "Risk Analysis",
        "Time Estimate",
        "Gap Analysis",        
    ]

    with open(file_path, mode="w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')

        writer.writeheader()

        for case in processed_use_cases:
            row = {fieldname: case.get(fieldname, "N/A") for fieldname in fieldnames}
            writer.writerow(row)

    print(f"CSV report created: {file_path}")


generate_csv_report(processed_use_cases)